# Active Appeals CCD MVP Base Build (Bronze Layer)
<table style='float:left;'>
   <tbody>
      <tr>
         <td style='text-align: left;'><b>Name: </b></td>
         <td>BRONZE_ACTIVE_APPEALS</td>
      </tr>
      <tr>
         <td style='text-align: left;'><b>Description: </b></td>
         <td>Notebook responsible for the bronze layer, common for all active appeal states..</td>
      </tr>
      <tr>
         <td style='text-align: left;'><b>First Created: </b></td>
         <td>May-2025</td>
      </tr>
      <tr>
         <th style='text-align: left;'><b>Changelog (JIRA ref/initials/date):</b></th>
         <th>Comments</th>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-634">ARIADM-634</a>/NSA/JUN-2025</td>
         <td>Active CCD MVP Base Build: Create Raw Bronze Tables from Landing Data</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-650">ARIADM-634</a>/NSA/JUN-2025</td>
         <td>Active CCD MVP Base Build: Add CSVs for ref tables: countries_nationality, countries_postal, countries_countryFromAddress, hearing_centres, reason_link and remissions</td>
      </tr>
       <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-665">ARIADM-634</a>/GB/JUN-2025</td>
         <td>cerate bronze 'default values' table from csv</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-665">ARIADM-634</a>/GB/JUN-2025</td>
         <td>cerate bronze 'default values' table from csv.</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-651">ARIADM-651</a>/NSA/JUL-2024</td>
         <td>Add explicit schema for lookUps : countries_nationality, countries_postal, countries_countryFromAddress, hearing_centres, reason_link and remissions</td>
      </tr>
   </tbody>
</table>

In [0]:
spark.conf.set("pipelines.tableManagedByMultiplePipelinesCheck.enabled", "false")

### Import packages

In [0]:
import dlt
import json
from pyspark.sql.functions import when, col,coalesce, current_timestamp, lit, date_format
from pyspark.sql.functions import *
from pyspark.sql.types import *
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
from pyspark.sql.window import Window

## Set Variables

In [0]:
config = spark.read.option("multiline", "true").json("dbfs:/configs/config.json")
env_name = config.first()["env"].strip().lower()
lz_key = config.first()["lz_key"].strip().lower()

print(f"env_code: {lz_key}")  # This won't be redacted
print(f"env_name: {env_name}")  # This won't be redacted

KeyVault_name = f"ingest{lz_key}-meta002-{env_name}"
print(f"KeyVault_name: {KeyVault_name}") 

In [0]:
# Service principal credentials
client_id = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-CLIENT-ID")
client_secret = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-CLIENT-SECRET")
tenant_id = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-TENANT-ID")

# Storage account names
curated_storage = f"ingest{lz_key}curated{env_name}"
checkpoint_storage = f"ingest{lz_key}xcutting{env_name}"
raw_storage = f"ingest{lz_key}raw{env_name}"
landing_storage = f"ingest{lz_key}landing{env_name}"
external_storage = f"ingest{lz_key}external{env_name}"


# Spark config for curated storage (Delta table)
spark.conf.set(f"fs.azure.account.auth.type.{curated_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{curated_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{curated_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{curated_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{curated_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{checkpoint_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{checkpoint_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{checkpoint_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{checkpoint_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{checkpoint_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{raw_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{raw_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{raw_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{raw_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{raw_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{landing_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{landing_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{landing_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{landing_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{landing_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")


# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{external_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{external_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{external_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{external_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{external_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

In [0]:
# read_hive = False

# Setting variables for use in subsequent cells
raw_path = f"abfss://raw@ingest{lz_key}raw{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS"
landing_path = f"abfss://landing@ingest{lz_key}landing{env_name}.dfs.core.windows.net/SQLServer/Sales/IRIS/dbo/"
bronze_path = f"abfss://bronze@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS"
silver_path = f"abfss://silver@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS"
gold_path = f"abfss://gold@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS"
external_path = f"abfss:///external-csv@ingest{lz_key}external{env_name}.dfs.core.windows.net/"
gold_outputs = "ARIADM/CCD/APPEALS"
hive_schema = "ariadm_ccd_apl"
# key_vault = "ingest00-keyvault-sbox"

html_path = f"abfss://html-template@ingest{lz_key}landing{env_name}.dfs.core.windows.net/"

# Print all variables
variables = {
    # "read_hive": read_hive,
    "raw_path": raw_path,
    "landing_path": landing_path,
    "bronze_path": bronze_path,
    "silver_path": silver_path,
    "gold_path": gold_path,
    "html_path": html_path,
    "gold_outputs": gold_outputs,
    "hive_schema": hive_schema,
    "key_vault": KeyVault_name
}

display(variables)

## Functions to Read Latest Landing Files

In [0]:

# Function to recursively list all files in the ADLS directory
def deep_ls(path: str, depth: int = 0, max_depth: int = 10) -> list:
    """
    Recursively list all files and directories in ADLS directory.
    Returns a list of all paths found.
    """
    output = set()  # Using a set to avoid duplicates
    if depth > max_depth:
        return list(output)

    try:
        children = dbutils.fs.ls(path)
        for child in children:
            if child.path.endswith(".parquet"):
                output.add(child.path.strip())  # Add only .parquet files to the set

            if child.isDir:
                output.update(deep_ls(child.path, depth=depth + 1, max_depth=max_depth))

    except Exception as e:
        print(f"Error accessing {path}: {e}")

    return list(output)

# Main function to read the latest parquet file, add audit columns, and return the DataFrame
def read_latest_parquet(folder_name: str, view_name: str, process_name: str, base_path: str = f"{landing_path}/") -> "DataFrame":
    """
    Reads the latest .parquet file from a specified folder, adds audit columns, creates a temporary Spark view, and returns the DataFrame.
    
    Parameters:
    - folder_name (str): The name of the folder to look for the .parquet files (e.g., "AdjudicatorRole").
    - view_name (str): The name of the temporary view to create (e.g., "tv_AdjudicatorRole").
    - process_name (str): The name of the process adding the audit information (e.g., "ARIA_ARM_JOH").
    - base_path (str): The base path for the folders in the data lake.
    
    Returns:
    - DataFrame: The DataFrame created from the latest .parquet file with added audit columns.
    """
    # Construct the full folder path
    folder_path = f"{base_path}{folder_name}/full/"
    
    # List all .parquet files in the folder
    all_files = deep_ls(folder_path)
    
    # Check if files were found
    if not all_files:
        print(f"No .parquet files found in {folder_path}")
        return None

    # Create a DataFrame from the file paths
    file_df = spark.createDataFrame([(f,) for f in all_files], ["file_path"])
    
    # Extract timestamp from the file name using a regex pattern (assuming it's the last underscore-separated part before ".parquet")
    file_df = file_df.withColumn("timestamp", regexp_extract("file_path", r"_(\d+)\.parquet$", 1).cast("long"))
    
    # Find the maximum timestamp
    max_timestamp = file_df.agg(max("timestamp")).collect()[0][0]
    
    # Filter to get the file with the maximum timestamp
    latest_file_df = file_df.filter(col("timestamp") == max_timestamp)
    latest_file = latest_file_df.first()["file_path"]
    
    # Print the latest file being loaded for logging purposes
    print(f"Reading latest file: {latest_file}")
    
    # Read the latest .parquet file into a DataFrame
    df = spark.read.option("inferSchema", "true").parquet(latest_file)
    
    # Add audit columns
    df = df.withColumn("AdtclmnFirstCreatedDatetime", current_timestamp()) \
           .withColumn("AdtclmnModifiedDatetime", current_timestamp()) \
           .withColumn("SourceFileName", lit(latest_file)) \
           .withColumn("InsertedByProcessName", lit(process_name))
    
    # Create or replace a temporary view
    df.createOrReplaceTempView(view_name)
    
    print(f"Loaded the latest file for {folder_name} into view {view_name} with audit columns")
    
    # Return the DataFrame
    return df


## Raw DLT Tables Creation

In [0]:
@dlt.table(
    name="raw_appealcase",
    comment="Delta Live Table ARIA AppealCase.",
    path=f"{raw_path}/Raw_AppealCase"
)
def Raw_AppealCase():
    return read_latest_parquet("AppealCase", "tv_AppealCase", "ARIA_ACTIVE_APPEALS")

@dlt.table(
    name="raw_caserep",
    comment="Delta Live Table ARIA CaseRep.",
    path=f"{raw_path}/Raw_CaseRep"
)
def raw_CaseRep():
     return read_latest_parquet("CaseRep", "tv_CaseRep", "ARIA_ACTIVE_APPEALS") 
 

@dlt.table(
    name="raw_representative",
    comment="Delta Live Table ARIA Representative.",
    path=f"{raw_path}/Raw_Representative"
)
def raw_Representative():
     return read_latest_parquet("Representative", "tv_Representative", "ARIA_ACTIVE_APPEALS")

@dlt.table(
    name="raw_casesponsor",
    comment="Delta Live Table ARIA CaseSponsor.",
    path=f"{raw_path}/Raw_CaseSponsor"
)
def raw_CaseSponsor():
     return read_latest_parquet("CaseSponsor", "tv_CaseSponsor", "ARIA_ACTIVE_APPEALS") 
 
@dlt.table(
    name="raw_caserespondent",
    comment="Delta Live Table ARIA CaseRespondent.",
    path=f"{raw_path}/Raw_CaseRespondent"
)
def CaseRespondent():
    return read_latest_parquet("CaseRespondent", "tv_CaseRespondent", "ARIA_ACTIVE_APPEALS")

@dlt.table(
    name="raw_filelocation",
    comment="Delta Live Table ARIA FileLocation.",
    path=f"{raw_path}/Raw_FileLocation"
)
def raw_FileLocation():
     return read_latest_parquet("FileLocation", "tv_FileLocation", "ARIA_ACTIVE_APPEALS") 
 
@dlt.table(
    name="raw_casefeesummary",
    comment="Delta Live Table ARIA CaseFeeSummary.",
    path=f"{raw_path}/Raw_CaseFeeSummary"
)
def raw_CaseFeeSummary():
    return read_latest_parquet("CaseFeeSummary", "tv_CaseFeeSummary", "ARIA_ACTIVE_APPEALS")  
 
@dlt.table(
    name="raw_caseappellant",
    comment="Delta Live Table ARIA CaseAppellant.",
    path=f"{raw_path}/Raw_CaseAppellant"
)
def raw_CaseAppellant():
     return read_latest_parquet("CaseAppellant", "tv_CaseAppellant", "ARIA_ACTIVE_APPEALS")
 
@dlt.table(
    name="raw_appellant",
    comment="Delta Live Table ARIA Appellant.",
    path=f"{raw_path}/Raw_Appellant"
)
def raw_Appellant():
     return read_latest_parquet("Appellant", "tv_Appellant", "ARIA_ACTIVE_APPEALS") 
 
@dlt.table(
    name="raw_transaction",
    comment="Delta Live Table ARIA Transaction.",
    path=f"{raw_path}/Raw_Transaction"
)
def raw_Transaction():
     return read_latest_parquet("Transaction", "tv_Transaction", "ARIA_ACTIVE_APPEALS")   

@dlt.table(
    name="raw_transactiontype",
    comment="Delta Live Table ARIA TransactionType.",
    path=f"{raw_path}/Raw_TransactionType"
)
def raw_TransactionType():
     return read_latest_parquet("TransactionType", "tv_TransactionType", "ARIA_ACTIVE_APPEALS")  
 
@dlt.table(
    name="raw_link",
    comment="Delta Live Table ARIA Link.",
    path=f"{raw_path}/Raw_Link"
)
def raw_Link():
     return read_latest_parquet("Link", "tv_Link", "ARIA_ACTIVE_APPEALS")  
 
@dlt.table(
    name="raw_linkdetail",
    comment="Delta Live Table ARIA LinkDetail.",
    path=f"{raw_path}/Raw_LinkDetail"
)
def raw_LinkDetail():
     return read_latest_parquet("LinkDetail", "tv_LinkDetail", "ARIA_ACTIVE_APPEALS")  
 
@dlt.table(
    name="raw_caseadjudicator",
    comment="Delta Live Table ARIA AppealTypeCategory.",
    path=f"{raw_path}/raw_caseadjudicator"
)
def raw_caseadjudicator():
     return read_latest_parquet("CaseAdjudicator", "tv_caseadjudicator", "ARIA_ACTIVE_APPEALS") 
 
@dlt.table(
    name="raw_adjudicator",
    comment="Delta Live Table ARIA Adjudicator.",
    path=f"{raw_path}/Raw_Adjudicator"
)
def raw_Adjudicator():
     return read_latest_parquet("Adjudicator", "tv_Adjudicator", "ARIA_ACTIVE_APPEALS")
 
@dlt.table(
    name="raw_appealcategory",
    comment="Delta Live Table ARIA AppealCategory.",
    path=f"{raw_path}/Raw_AppealCategory"
)
def raw_AppealCategory():
     return read_latest_parquet("AppealCategory", "tv_AppealCategory", "ARIA_ACTIVE_APPEALS") 
 
@dlt.table(
    name="raw_documentsreceived",
    comment="Delta Live Table ARIA DocumentsReceived.",
    path=f"{raw_path}/Raw_DocumentsReceived"
)
def raw_DocumentsReceived():
     return read_latest_parquet("DocumentsReceived", "tv_DocumentsReceived", "ARIA_ACTIVE_APPEALS")  
 
@dlt.table(
    name="raw_history",
    comment="Delta Live Table ARIA History.",
    path=f"{raw_path}/Raw_History"
)
def raw_History():
     return read_latest_parquet("History", "tv_History", "ARIA_ACTIVE_APPEALS")
 

@dlt.table(
    name="raw_caselist",
    comment="Delta Live Table ARIA CaseList.",
    path=f"{raw_path}/Raw_CaseList"
)
def raw_CaseList():
     return read_latest_parquet("CaseList", "tv_CaseList", "ARIA_ARM_APPEALS") 
 
@dlt.table(
    name="raw_status",
    comment="Delta Live Table ARIA Status.",
    path=f"{raw_path}/Raw_Status"
)
def raw_Status():
     return read_latest_parquet("Status", "tv_Status", "ARIA_ARM_APPEALS") 
 
@dlt.table(
    name="raw_list",
    comment="Delta Live Table ARIA Status.",
    path=f"{raw_path}/raw_list"
)
def raw_list():
     return read_latest_parquet("List", "tv_List", "ARIA_ACTIVE_APPEALS") 
 

@dlt.table(
    name="raw_listtype",
    comment="Delta Live Table ARIA Status.",
    path=f"{raw_path}/raw_listtype"
)
def raw_listtype():
     return read_latest_parquet("ListType", "tv_ListType", "ARIA_ACTIVE_APPEALS") 
 
@dlt.table(
    name="raw_hearingtype",
    comment="Delta Live Table ARIA Status.",
    path=f"{raw_path}/raw_hearingtype"
)
def raw_hearingtype():
     return read_latest_parquet("HearingType", "tv_HearingType", "ARIA_ACTIVE_APPEALS") 
 
@dlt.table(
    name="raw_court",
    comment="Delta Live Table ARIA Status.",
    path=f"{raw_path}/raw_court"
)
def raw_court():
     return read_latest_parquet("Court", "tv_Court", "ARIA_ACTIVE_APPEALS")


@dlt.table(
    name="raw_listsitting",
    comment="Delta Live Table ARIA Status.",
    path=f"{raw_path}/raw_listsitting"
)
def raw_listsitting():
     return read_latest_parquet("ListSitting", "tv_ListSitting", "ARIA_ACTIVE_APPEALS") 
 

## Bronze DLT Tables Creation

### Transformation M1 bronze_appealcase_crep_rep_floc_cspon_cfs

```sql
-- m1.bronze_appealcase_crep_rep_floc_cspon_cfs
SELECT
-- AppealCase
ac.CaseNo,
ac.CasePrefix,
ac.OutOfTimeIssue,
ac.DateLodged,
ac.DateAppealReceived,
ac.CentreId,
ac.NationalityId,
ac.AppealTypeId,
ac.DeportationDate,
ac.RemovalDate,
ac.VisitVisaType,
ac.DateOfApplicationDecision,
ac.HORef,
ac.InCamera,
ac.CourtPreference,
ac.LanguageId,
ac.Interpreter,
-- CaseRep
crep.RepresentativeId,
crep.Name AS CaseRepName,
crep.Address1 AS CaseRepAddress1,
crep.Address2 AS CaseRepAddress2,
crep.Address3 AS CaseRepAddress3,
crep.Address4 AS CaseRepAddress4,
crep.Address5 AS CaseRepAddress5,
crep.Postcode AS CaseRepPostcode,
crep.Contact,
crep.Email AS CaseRepEmail,
crep.FileSpecificEmail,
-- Representative
rep.Name AS RepName,
rep.Address1 AS RepAddress1,
rep.Address2 AS RepAddress2,
rep.Address3 AS RepAddress3,
rep.Address4 AS RepAddress4,
rep.Address5 AS RepAddress5,
rep.Postcode AS RepPostcode,
rep.Email AS RepEmail,
-- CaseSponsor
cspon.Name AS SponsorName,
cspon.Forenames AS SponsorForenames,
cspon.Address1 AS SponsorAddress1,
cspon.Address2 AS SponsorAddress2,
cspon.Address3 AS SponsorAddress3,
cspon.Address4 AS SponsorAddress4,
cspon.Address5 AS SponsorAddress5,
cspon.Postcode AS SponsorPostcode,
cspon.Email AS SponsorEmail,
cspon.Telephone AS SponsorTelephone,
cspon.Authorised AS SponsorAuthorisation,
-- CaseRespondent
cr.MainRespondentId,
-- FileLocation
floc.DeptId,
-- CaseFeeSummary
cfs.PaymentRemissionRequested,
cfs.PaymentRemissionReason,
cfs.PaymentRemissionGranted,
cfs.PaymentRemissionReasonNote,
cfs.LSCReference,
cfs.ASFReferenceNo,
cfs.DateCorrectFeeReceived
FROM dbo.AppealCase ac
LEFT OUTER JOIN dbo.CaseRep crep ON ac.CaseNo = crep.CaseNo
LEFT OUTER JOIN dbo.Representative rep ON crep.RepresentativeId = rep.RepresentativeId
LEFT OUTER JOIN dbo.CaseSponsor cspon ON cspon.CaseNo = ac.CaseNo
LEFT OUTER JOIN dbo.CaseRespondent cr on ac.CaseNo = cr.CaseNo
LEFT OUTER JOIN dbo.FileLocation floc ON floc.CaseNo = ac.CaseNo
LEFT OUTER JOIN dbo.CaseFeeSummary cfs ON cfs.CaseNo = ac.CaseNo
```

In [0]:
@dlt.table(
    name="bronze_appealcase_crep_rep_floc_cspon_cfs",
    comment="Delta Live Table combining AppealCase with CaseRep, Representative, CaseSponsor, CaseRespondent, FileLocation, and CaseFeeSummary data.",
    path=f"{bronze_path}/bronze_appealcase_crep_rep_floc_cspon_cfs"
)
def bronze_appealcase_crep_rep_floc_cspon_cfs():
    df = (
        dlt.read("raw_appealcase").alias("ac")
        .join(dlt.read("raw_caserep").alias("crep"), col("ac.CaseNo") == col("crep.CaseNo"), "left_outer")
        .join(dlt.read("raw_representative").alias("rep"), col("crep.RepresentativeId") == col("rep.RepresentativeId"), "left_outer")
        .join(dlt.read("raw_casesponsor").alias("cspon"), col("ac.CaseNo") == col("cspon.CaseNo"), "left_outer")
        .join(dlt.read("raw_caserespondent").alias("cr"), col("ac.CaseNo") == col("cr.CaseNo"), "left_outer")
        .join(dlt.read("raw_filelocation").alias("floc"), col("ac.CaseNo") == col("floc.CaseNo"), "left_outer")
        .join(dlt.read("raw_casefeesummary").alias("cfs"), col("ac.CaseNo") == col("cfs.CaseNo"), "left_outer")
        .select(
            # AppealCase
            col("ac.CaseNo"),
            col("ac.CasePrefix"),
            col("ac.OutOfTimeIssue"),
            col("ac.DateLodged"),
            col("ac.DateAppealReceived"),
            col("ac.CentreId"),
            col("ac.NationalityId"),
            col("ac.AppealTypeId"),
            col("ac.DeportationDate"),
            col("ac.RemovalDate"),
            col("ac.VisitVisaType"),
            col("ac.DateOfApplicationDecision"),
            col("ac.HORef"),
            col("ac.InCamera"),
            col("ac.CourtPreference"),
            col("ac.LanguageId"),
            col("ac.Interpreter"),

            # CaseRep
            col("crep.RepresentativeId"),
            col("crep.Name").alias("CaseRepName"),
            col("crep.Address1").alias("CaseRepAddress1"),
            col("crep.Address2").alias("CaseRepAddress2"),
            col("crep.Address3").alias("CaseRepAddress3"),
            col("crep.Address4").alias("CaseRepAddress4"),
            col("crep.Address5").alias("CaseRepAddress5"),
            col("crep.Postcode").alias("CaseRepPostcode"),
            col("crep.Contact"),
            col("crep.Email").alias("CaseRepEmail"),
            col("crep.FileSpecificEmail"),

            # Representative
            col("rep.Name").alias("RepName"),
            col("rep.Address1").alias("RepAddress1"),
            col("rep.Address2").alias("RepAddress2"),
            col("rep.Address3").alias("RepAddress3"),
            col("rep.Address4").alias("RepAddress4"),
            col("rep.Address5").alias("RepAddress5"),
            col("rep.Postcode").alias("RepPostcode"),
            col("rep.Email").alias("RepEmail"),

            # CaseSponsor
            col("cspon.Name").alias("SponsorName"),
            col("cspon.Forenames").alias("SponsorForenames"),
            col("cspon.Address1").alias("SponsorAddress1"),
            col("cspon.Address2").alias("SponsorAddress2"),
            col("cspon.Address3").alias("SponsorAddress3"),
            col("cspon.Address4").alias("SponsorAddress4"),
            col("cspon.Address5").alias("SponsorAddress5"),
            col("cspon.Postcode").alias("SponsorPostcode"),
            col("cspon.Email").alias("SponsorEmail"),
            col("cspon.Telephone").alias("SponsorTelephone"),
            col("cspon.Authorised").alias("SponsorAuthorisation"),

            # CaseRespondent
            col("cr.MainRespondentId"),

            # FileLocation
            col("floc.DeptId"),

            # CaseFeeSummary
            col("cfs.PaymentRemissionRequested"),
            col("cfs.PaymentRemissionReason"),
            col("cfs.PaymentRemissionGranted"),
            col("cfs.PaymentRemissionReasonNote"),
            col("cfs.LSCReference"),
            col("cfs.ASFReferenceNo"),
            col("cfs.DateCorrectFeeReceived")
        )
    )

    return df


### Transformation M2 bronze_appealcase_caseappellant_appellant

```sql
-- m2.bronze_appealcase_caseappellant_appellant 
-- This query *could* pull back multiple rows per case however for all the cases migrating to CCD there is only ever 1 row per CaseNo. 

SELECT 
ac.CaseNo, 
-- Appellant 
ap.Name AS AppellantName, 
ap.Forenames AS AppellantForenames, 
ap.BirthDate, 
ap.Email AS AppellantEmail, 
ap.Telephone AS AppellantTelephone, 
ap.Address1 AS AppellantAddress1, 
ap.Address2 AS AppellantAddress2, 
ap.Address3 AS AppellantAddress3, 
ap.Address4 AS AppellantAddress4, 
ap.Address5 AS AppellantAddress5, 
ap.Postcode AS AppellantPostcode, 
ap.AppellantCountryId, 
ap.FCONumber 
FROM dbo.AppealCase ac 
LEFT OUTER JOIN dbo.CaseAppellant cap ON cap.CaseNO = ac.CaseNo 
LEFT OUTER JOIN dbo.Appellant ap ON cap.AppellantId = ap.AppellantId 
```

In [0]:
@dlt.table(
    name="bronze_appealcase_caseappellant_appellant",
    comment="DLT table joining AppealCase with CaseAppellant and Appellant details.",
    path=f"{bronze_path}/bronze_appealcase_caseappellant_appellant"
)
def bronze_appealcase_caseappellant_appellant():
    df = (
        dlt.read("raw_appealcase").alias("ac")
        .join(dlt.read("raw_caseappellant").alias("cap"), col("ac.CaseNo") == col("cap.CaseNo"), "left_outer")
        .join(dlt.read("raw_appellant").alias("ap"), col("cap.AppellantId") == col("ap.AppellantId"), "left_outer")
        .select(
            col("ac.CaseNo"),

            # Appellant
            col("ap.Name").alias("AppellantName"),
            col("ap.Forenames").alias("AppellantForenames"),
            col("ap.BirthDate"),
            col("ap.Email").alias("AppellantEmail"),
            col("ap.Telephone").alias("AppellantTelephone"),
            col("ap.Address1").alias("AppellantAddress1"),
            col("ap.Address2").alias("AppellantAddress2"),
            col("ap.Address3").alias("AppellantAddress3"),
            col("ap.Address4").alias("AppellantAddress4"),
            col("ap.Address5").alias("AppellantAddress5"),
            col("ap.Postcode").alias("AppellantPostcode"),
            col("ap.AppellantCountryId"),
            col("ap.FCONumber")
        )
    )

    return df


### Transformation M3 bronze_status_htype_clist_list_ltype_court_lsitting_adj

```sql
-- m3.bronze_status_htype_clist_list_ltype_court_lsitting_adj 
-- This will return multiple rows per CaseNo  
SELECT  
s.StatusId, 
s.CaseNo, 
s.CaseStatus, 
s.Outcome, 
s.KeyDate AS HearingDate, 
s.CentreId, 
s.DecisionDate, 
s.Party, 
s.DateReceived, 
s.OutOfTime, 
s.DecisionReserved, 
s.AdjudicatorId, 
adjs.Surname AS AdjSurname,  
adjs.Forenames AS AdjForenames,  
adjs.Title AS AdjTitle,  
s.Promulgated AS DateOfService, 
s.AdditionalLanguageId, 
-- Listing details fields 
s.ListedCentre AS HearingCentre, 
c.CourtName, 
l.ListName, 
lt.Description AS ListType, 
ht.Description AS HearingType, 
cl.StartTime, 
cl.TimeEstimate, 
-- Assigned Judges & Clerk for Listing Details 
adjLS1.Surname AS Judge1FTSurname, 
adjLS1.Forenames AS Judge1FTForenames, 
adjLS1.Title AS Judge1FTTitle,	 
adjLS2.Surname AS Judge2FTSurname, 
adjLS2.Forenames AS Judge2FTForenames, 
adjLS2.Title AS Judge2FTTitle,	 
adjLS3.Surname AS Judge3FTSurname, 
adjLS3.Forenames AS Judge3FTForenames, 
adjLS3.Title AS Judge3FTTitle,	 
adjLS4.Surname AS CourtClerkSurname, 
adjLS4.Forenames AS CourtClerkForenames, 
adjLS4.Title AS CourtClerkTitle, 
ac.Notes 
FROM dbo.Status s 
LEFT OUTER JOIN dbo.CaseList cl ON cl.StatusId = s.StatusId 
LEFT OUTER JOIN dbo.List l ON l.ListId = cl.ListId 
LEFT OUTER JOIN dbo.HearingType ht ON cl.HearingTypeId = ht.HearingTypeId 
LEFT OUTER JOIN dbo.Adjudicator adjs ON s.AdjudicatorId = adjs.AdjudicatorId 
LEFT OUTER JOIN dbo.ListType lt ON l.ListTypeId = lt.ListTypeId 
LEFT OUTER JOIN dbo.Court c ON c.CourtId = l.CourtId 
LEFT OUTER JOIN dbo.AppealCase ac ON ac.CaseNO = s.CaseNo 
LEFT OUTER JOIN dbo.ListSitting LS1 ON l.ListId = LS1.ListId AND LS1.Position = 10 AND LS1.Cancelled = 0 
LEFT OUTER JOIN dbo.Adjudicator adjLS1 ON LS1.AdjudicatorId = adjLS1.AdjudicatorId 
LEFT OUTER JOIN dbo.ListSitting LS2 ON l.ListId = LS2.ListId AND LS2.Position = 11 AND LS2.Cancelled = 0 
LEFT OUTER JOIN dbo.Adjudicator adjLS2 ON LS2.AdjudicatorId = adjLS2.AdjudicatorId 
LEFT OUTER JOIN dbo.ListSitting LS3 ON l.ListId = LS3.ListId AND LS3.Position = 12 AND LS3.Cancelled = 0 
LEFT OUTER JOIN dbo.Adjudicator adjLS3 ON LS3.AdjudicatorId = adjLS3.AdjudicatorId 
LEFT OUTER JOIN dbo.ListSitting LS4 ON l.ListId = LS4.ListId AND LS4.Position = 3 AND LS4.Cancelled = 0 
LEFT OUTER JOIN dbo.Adjudicator adjLS4 ON LS4.AdjudicatorId = adjLS4.AdjudicatorId

```

In [0]:
@dlt.table(
    name="bronze_status_htype_clist_list_ltype_court_lsitting_adj",
    comment="DLT table joining Status with hearing types, court listings, adjudicators, and associated metadata.",
    path=f"{bronze_path}/bronze_status_htype_clist_list_ltype_court_lsitting_adj"
)
def bronze_status_htype_clist_list_ltype_court_lsitting_adj():
    df = (
        dlt.read("raw_status").alias("s")
        .join(dlt.read("raw_caselist").alias("cl"), col("s.StatusId") == col("cl.StatusId"), "left_outer")
        .join(dlt.read("raw_list").alias("l"), col("cl.ListId") == col("l.ListId"), "left_outer")
        .join(dlt.read("raw_hearingtype").alias("ht"), col("cl.HearingTypeId") == col("ht.HearingTypeId"), "left_outer")
        .join(dlt.read("raw_adjudicator").alias("adjs"), col("s.AdjudicatorId") == col("adjs.AdjudicatorId"), "left_outer")
        .join(dlt.read("raw_listtype").alias("lt"), col("l.ListTypeId") == col("lt.ListTypeId"), "left_outer")
        .join(dlt.read("raw_court").alias("c"), col("l.CourtId") == col("c.CourtId"), "left_outer")
        .join(dlt.read("raw_appealcase").alias("ac"), col("s.CaseNo") == col("ac.CaseNo"), "left_outer")
        .join(dlt.read("raw_listsitting").alias("LS1"), (col("l.ListId") == col("LS1.ListId")) & (col("LS1.Position") == 10) & (col("LS1.Cancelled") == 0), "left_outer")
        .join(dlt.read("raw_adjudicator").alias("adjLS1"), col("LS1.AdjudicatorId") == col("adjLS1.AdjudicatorId"), "left_outer")
        .join(dlt.read("raw_listsitting").alias("LS2"), (col("l.ListId") == col("LS2.ListId")) & (col("LS2.Position") == 11) & (col("LS2.Cancelled") == 0), "left_outer")
        .join(dlt.read("raw_adjudicator").alias("adjLS2"), col("LS2.AdjudicatorId") == col("adjLS2.AdjudicatorId"), "left_outer")
        .join(dlt.read("raw_listsitting").alias("LS3"), (col("l.ListId") == col("LS3.ListId")) & (col("LS3.Position") == 12) & (col("LS3.Cancelled") == 0), "left_outer")
        .join(dlt.read("raw_adjudicator").alias("adjLS3"), col("LS3.AdjudicatorId") == col("adjLS3.AdjudicatorId"), "left_outer")
        .join(dlt.read("raw_listsitting").alias("LS4"), (col("l.ListId") == col("LS4.ListId")) & (col("LS4.Position") == 3) & (col("LS4.Cancelled") == 0), "left_outer")
        .join(dlt.read("raw_adjudicator").alias("adjLS4"), col("LS4.AdjudicatorId") == col("adjLS4.AdjudicatorId"), "left_outer")
        .select(
            col("s.StatusId"),
            col("s.CaseNo"),
            col("s.CaseStatus"),
            col("s.Outcome"),
            col("s.KeyDate").alias("HearingDate"),
            col("s.CentreId"),
            col("s.DecisionDate"),
            col("s.Party"),
            col("s.DateReceived"),
            col("s.OutOfTime"),
            col("s.DecisionReserved"),
            col("s.AdjudicatorId"),
            col("adjs.Surname").alias("AdjSurname"),
            col("adjs.Forenames").alias("AdjForenames"),
            col("adjs.Title").alias("AdjTitle"),
            col("s.Promulgated").alias("DateOfService"),
            col("s.AdditionalLanguageId"),
            col("s.ListedCentre").alias("HearingCentre"),
            col("c.CourtName"),
            col("l.ListName"),
            col("lt.Description").alias("ListType"),
            col("ht.Description").alias("HearingType"),
            col("cl.StartTime"),
            col("cl.TimeEstimate"),
            col("adjLS1.Surname").alias("Judge1FTSurname"),
            col("adjLS1.Forenames").alias("Judge1FTForenames"),
            col("adjLS1.Title").alias("Judge1FTTitle"),
            col("adjLS2.Surname").alias("Judge2FTSurname"),
            col("adjLS2.Forenames").alias("Judge2FTForenames"),
            col("adjLS2.Title").alias("Judge2FTTitle"),
            col("adjLS3.Surname").alias("Judge3FTSurname"),
            col("adjLS3.Forenames").alias("Judge3FTForenames"),
            col("adjLS3.Title").alias("Judge3FTTitle"),
            col("adjLS4.Surname").alias("CourtClerkSurname"),
            col("adjLS4.Forenames").alias("CourtClerkForenames"),
            col("adjLS4.Title").alias("CourtClerkTitle"),
            col("ac.Notes")
        )
    )

    return df

### Transformation M4 bronze_appealcase_transaction_transactiontype

```sql
-- m4.bronze_appealcase_transaction_transactiontype
-- This will return multiple rows per CaseNo
SELECT
-- Transaction
t.CaseNo,
t.TransactionId,
t.TransactionTypeId,
t.ReferringTransactionId,
t.Amount,
t.TransactionDate,
t.Status,
-- TransactionType
tt.SumBalance,
tt.SumTotalFee,
tt.SumTotalPay
FROM dbo.[Transaction] t
LEFT OUTER JOIN dbo.TransactionType tt ON t.TransactionTypeId = tt.TransactionTypeId
```

In [0]:
@dlt.table(
    name="bronze_appealcase_transaction_transactiontype",
    comment="DLT table joining Transaction with TransactionType to return multiple rows per CaseNo.",
    path=f"{bronze_path}/bronze_appealcase_transaction_transactiontype"
)
def bronze_appealcase_transaction_transactiontype():
    df = (
        dlt.read("raw_transaction").alias("t")
        .join(
            dlt.read("raw_transactiontype").alias("tt"),
            col("t.TransactionTypeId") == col("tt.TransactionTypeId"),
            "left_outer"
        )
        .select(
            col("t.CaseNo"),
            col("t.TransactionId"),
            col("t.TransactionTypeId"),
            col("t.ReferringTransactionId"),
            col("t.Amount"),
            col("t.TransactionDate"),
            col("t.Status"),
            # TransactionType 
            col("tt.SumBalance"),
            col("tt.SumTotalFee"),
            col("tt.SumTotalPay")
        )
    )

    return df



### Transformation M5 bronze_appealcase_transaction_transactiontype

```sql
-- m5.bronze_appealcase_link_linkdetail
-- This can return multiple rows per CaseNo
SELECT
-- Link
l.CaseNo,
l.LinkNo,
-- LinkDetail
ld.ReasonLinkId
FROM dbo.Link l
LEFT OUTER JOIN dbo.LinkDetail ld ON l.LinkNo = ld.LinkNo
ORDER BY CaseNO
```

In [0]:
@dlt.table(
    name="bronze_appealcase_link_linkdetail",
    comment="DLT table joining Link with LinkDetail. Can return multiple rows per CaseNo.",
    path=f"{bronze_path}/bronze_appealcase_link_linkdetail"
)
def bronze_appealcase_link_linkdetail():
    df = (
        dlt.read("raw_link").alias("l")
        .join(
            dlt.read("raw_linkdetail").alias("ld"),
            col("l.LinkNo") == col("ld.LinkNo"),
            "left_outer"
        )
        .select(
            col("l.CaseNo"),
            col("l.LinkNo"),
            col("ld.ReasonLinkId")
        )
    )

    return df


### Transformation M6 bronze_caseadjudicator_adjudicator

In [0]:
@dlt.table(
    name="bronze_caseadjudicator_adjudicator",
    comment="DLT table joining CaseAdjudicator with Adjudicator. Can return multiple rows per CaseNo.",
    path=f"{bronze_path}/bronze_caseadjudicator_adjudicator"
)
def bronze_caseadjudicator_adjudicator():
    df = (
        dlt.read("raw_caseadjudicator").alias("ca")
        .join(
            dlt.read("raw_adjudicator").alias("adj"),
            (col("ca.AdjudicatorId") == col("adj.AdjudicatorId")) & (col("adj.DoNotList") == 0),
            "inner"
        )
        .select(
            col("ca.CaseNo"),
            col("ca.Required"),
            col("adj.Surname").alias("JudgeSurname"),
            col("adj.Forenames").alias("JudgeForenames"),
            col("adj.Title").alias("JudgeTitle")
        )
    )

    return df


### Transformation bronze_appealcategory

In [0]:
@dlt.table(
    name="bronze_appealcategory",
    comment="DLT table for AppealCategory. Returns multiple rows per CaseNo.",
    path=f"{bronze_path}/bronze_appealcategory"
)
def bronze_appealcategory():
    return (
        dlt.read("raw_appealcategory")
        .select(
            col("CaseNo"),
            col("CategoryId")
        )
    )


### Transformation bronze_documentsreceived

In [0]:
@dlt.table(
    name="bronze_documentsreceived",
    comment="DLT table for DocumentsReceived; may contain multiple rows per CaseNo.",
    path=f"{bronze_path}/bronze_documentsreceived"
)
def bronze_documentsreceived():
    return (
        dlt.read("raw_documentsreceived")      # adjust if your raw table name differs
            .select(
                col("CaseNo"),
                col("ReceivedDocumentId"),
                col("DateReceived")
            )
    )


### Transformation bronze_history

In [0]:
@dlt.table(
    name="bronze_history",
    comment="DLT table for History; may contain multiple rows per CaseNo.",
    path=f"{bronze_path}/bronze_history"
)
def bronze_history():
    return (
        dlt.read("raw_history")   # Replace with actual raw source name if different
            .select(
                col("HistoryId"),
                col("CaseNo"),
                col("HistType"),
                col("Comment")
            )
    )


## Bronze Reference DLT Tables Creation
> Note: Reference data from csv

### Transformation bronze_countries_nationality

In [0]:
@dlt.table(
    name="bronze_countries_nationality",
    comment="DLT table for History; may contain multiple rows per CaseNo.",
    path=f"{bronze_path}/bronze_countries_nationality"
)
def bronze_countries_nationality():
    schema = StructType([
        StructField("NationalityId", IntegerType(), True),
        StructField("Description", StringType(), True),
        StructField("countryCode", StringType(), True),
        StructField("appellantNationalitiesDescription", StringType(), True)
    ])
    
    return (
        spark.read.format("csv")
            .option("header", "true")
            .option("inferSchema", "false")
            .option("encoding", "UTF-8")
            .schema(schema)
            .load(f"abfss://external-csv@{external_storage}.dfs.core.windows.net/ReferenceData/countries_nationality.csv")
            .select("*")
    )

### Transformation bronze_countries_postal

In [0]:
@dlt.table(
    name="bronze_countries_postal",
    comment="DLT table for History; may contain multiple rows per CaseNo.",
    path=f"{bronze_path}/bronze_countries_postal"
)
def bronze_countries_postal():
    schema = StructType([
        StructField("AppellantCountryId", IntegerType(), True),
        StructField("Description", StringType(), True),
        StructField("countryGovUkOocAdminJ", StringType(), True)
    ])
    
    return (
        spark.read.format("csv")
            .option("header", "true")
            .option("inferSchema", "false")
            .option("encoding", "UTF-8")
            .schema(schema)
            .load(f"abfss://external-csv@{external_storage}.dfs.core.windows.net/ReferenceData/countries_postal.csv")
            .select("*")
    )

### Transformation bronze_countries_countryFromAddress

In [0]:
@dlt.table(
    name="bronze_countries_countryFromAddress",
    comment="DLT table for History; may contain multiple rows per CaseNo.",
    path=f"{bronze_path}/bronze_countries_countryFromAddress"
)
def bronze_countries_countryFromAddress():
    schema = StructType([
        StructField("CountryId", IntegerType(), True),
        StructField("Description", StringType(), True),
        StructField("countryCode", StringType(), True)
    ])
    
    return (
        spark.read.format("csv")
            .option("header", "true")
            .option("inferSchema", "false")
            .option("encoding", "UTF-8")
            .schema(schema)
            .load(f"abfss://external-csv@{external_storage}.dfs.core.windows.net/ReferenceData/countries_countryFromAddress.csv")
            .select("*")
    )

### Transformation bronze_hearing_centres

In [0]:
@dlt.table(
    name="bronze_hearing_centres",
    comment="DLT table for History; may contain multiple rows per CaseNo.",
    path=f"{bronze_path}/bronze_hearing_centres"
)
def bronze_hearing_centres():
    input_schema = StructType([
        StructField("CentreId", IntegerType(), True),
        StructField("prevFileLocation", StringType(), True),
        StructField("hearingCentre", StringType(), True),
        StructField("staffLocation", StringType(), True),
        StructField("caseManagementLocation", StringType(), True),
        StructField("locationCode", StringType(), True),
        StructField("locationLabel", StringType(), True),
        StructField("selectedHearingCentreRefData", StringType(), True),
        StructField("listCaseHearingCentre", StringType(), True),
        StructField("applicationChangeDesignatedHearingCentre", StringType(), True)
    ])

    caseManagementLocation_schema = StructType([
        StructField("region", StringType(), True),
        StructField("baseLocation", StringType(), True)
    ])

    df = (
        spark.read.format("csv")
            .option("header", "true")
            .option("inferSchema", "false")
            .option("encoding", "UTF-8")
            .option("multiline", "true")
            .option("escape", "\"")
            .schema(input_schema)
            .load(f"abfss://external-csv@{external_storage}.dfs.core.windows.net/ReferenceData/hearing_centres.csv")
            .withColumn("caseManagementLocation", regexp_replace(trim(regexp_replace(col("caseManagementLocation"), ",$", "")), "^\\s+|\\s+$", ""))  # Remove trailing commas and leading/trailing whitespace
            .withColumn("caseManagementLocation", from_json(col("caseManagementLocation"), caseManagementLocation_schema))
            .select("*")
    )

    return df

### Transformation bronze_remissions

In [0]:
@dlt.table(
    name="bronze_remissions",
    comment="DLT table for History; may contain multiple rows per CaseNo.",
    path=f"{bronze_path}/bronze_remissions"
)
def bronze_remissions():
    schema = StructType([
        StructField("PaymentRemissionRequested", StringType(), True),
        StructField("PaymentRemissionReason", StringType(), True),
        StructField("ReasonDescription", StringType(), True),
        StructField("remissionType", StringType(), True),
        StructField("remissionClaim", StringType(), True),
        StructField("feeRemissionType", StringType(), True),
        StructField("exceptionalCircumstances", StringType(), True),
        StructField("legalAidAccountNumber", StringType(), True),
        StructField("asylumSupportReference", StringType(), True),
        StructField("helpWithFeesReferenceNumber", StringType(), True)
    ])
    
    return (
        spark.read.format("csv")
            .option("header", "true")
            .option("inferSchema", "false")
            .option("encoding", "UTF-8")
            .option("multiline", "true")
            .option("escape", "\"")
            .schema(schema)
            .load(f"abfss://external-csv@{external_storage}.dfs.core.windows.net/ReferenceData/remissions.csv")
            .select("*")
    )

### Transformation bronze_appealtype

In [0]:
@dlt.table(
    name="bronze_appealtype",
    comment="DLT table for AppealType.",
    path=f"{bronze_path}/bronze_appealtype"
)
def bronze_appealtype():
    input_schema = StructType([
        StructField("CCDAppealType", StringType(), True),
        StructField("appealType", StringType(), True),
        StructField("hmctsCaseCategory", StringType(), True),
        StructField("appealTypeDescription", StringType(), True),
        StructField("caseManagementCategory", StringType(), True),
        StructField("appealReferenceNumber", StringType(), True),
        StructField("isAppealReferenceNumberAvailable", StringType(), True),
        StructField("ccdReferenceNumberForDisplay", StringType(), True)
    ])

    caseManagementCategory_json_schema = StructType([
        StructField("value", StructType([
            StructField("code", StringType(), True),
            StructField("label", StringType(), True)
        ]), True),
        StructField("list_items", ArrayType(
            StructType([
                StructField("code", StringType(), True),
                StructField("label", StringType(), True)
            ])
        ), True)
    ])

    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "false") \
        .option("encoding", "UTF-8") \
        .option("multiline", "true") \
        .option("escape", "\"") \
        .schema(input_schema) \
        .load(f"abfss://external-csv@{external_storage}.dfs.core.windows.net/ReferenceData/appeal_type.csv") \
        .withColumnRenamed("CCD Appeal Type (Conditions - Column H)", "CCDAppealType") \
        .withColumn("caseManagementCategory", regexp_replace(trim(regexp_replace(col("caseManagementCategory"), ",$", "")), "^\\s+|\\s+$", "")) \
        .select("*")

    # Parse JSON string column to struct
    df_parsed = df.withColumn("caseManagementCategoryParsed", from_json(col("caseManagementCategory"), caseManagementCategory_json_schema))

    # Apply schema to df_parsed
    df_final = df_parsed.select(
        col("CCDAppealType").cast(StringType()),
        col("appealType").cast(StringType()),
        col("hmctsCaseCategory").cast(StringType()),
        col("appealTypeDescription").cast(StringType()),
        col("caseManagementCategoryParsed").alias("caseManagementCategory"),
        col("appealReferenceNumber").cast(StringType()),
        col("isAppealReferenceNumberAvailable").cast(StringType()),
        col("ccdReferenceNumberForDisplay").cast(StringType())
    )

    return df_final

### Transformation bronze_statevalue

In [0]:
@dlt.table(
    name="bronze_statevalue",
    comment="DLT table for default state values.",
    path=f"{bronze_path}/bronze_statevalue"
)
def bronze_statevalue():
    schema = StructType([
        StructField("Field", StringType(), True),
        StructField("paymentPending", StringType(), True),
        StructField("appealSubmitted", StringType(), True),
        StructField("awaitingRespondentEvidence_a", StringType(), True),
        StructField("awaitingRespondentEvidence_b", StringType(), True),
        StructField("caseUnderReview", StringType(), True),
        StructField("reasonsForAppealSubmitted", StringType(), True),
        StructField("listing", StringType(), True)
    ])
    
    return (
        spark.read.format("csv")
        .option("header", "true")
        .option("inferSchema", "false")
        .option("encoding", "UTF-8")
        .option("multiline", "true")
        .option("escape", "\"")
        .schema(schema)
        .load(f"abfss://external-csv@{external_storage}.dfs.core.windows.net/ReferenceData/statevalue_mappings.csv")
        .select("*")
    )

display(bronze_appealtype)

## Exit Notebook with Success Message

In [0]:
dbutils.notebook.exit("Notebook completed successfully")

### Appendix

In [0]:
# %sql
# select * from ariadm_active_appeals.bronze_appealcase_crep_rep_floc_cspon_cfs

In [0]:
# config = spark.read.option("multiline", "true").json("dbfs:/configs/config.json")

# env = config.first()["env"].strip().lower()

# lz_key = config.first()["lz_key"].strip().lower()
 
# keyvault_name = f"ingest00-meta002-{env}"
 
# # Access the Service Principle secrets from keyvaults

# client_secret = dbutils.secrets.get(scope=keyvault_name, key='SERVICE-PRINCIPLE-CLIENT-SECRET')

# tenant_id = dbutils.secrets.get(scope=keyvault_name, key='SERVICE-PRINCIPLE-TENANT-ID')

# client_id = dbutils.secrets.get(scope=keyvault_name, key='SERVICE-PRINCIPLE-CLIENT-ID')
 
# storage_accounts = ["external"]
 
# for storage_account in storage_accounts:

#     spark.conf.set(

#         f'fs.azure.account.auth.type.ingest00{storage_account}{env}.dfs.core.windows.net', 'OAuth')
#     spark.conf.set(
#         f'fs.azure.account.oauth.provider.type.ingest00{storage_account}{env}.dfs.core.windows.net', 'org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider')
#     spark.conf.set(
#         f'fs.azure.account.oauth2.client.id.ingest00{storage_account}{env}.dfs.core.windows.net', client_id)
#     spark.conf.set(
#         f'fs.azure.account.oauth2.client.secret.ingest00{storage_account}{env}.dfs.core.windows.net', client_secret)
#     spark.conf.set(

#         f'fs.azure.account.oauth2.client.endpoint.ingest00{storage_account}{env}.dfs.core.windows.net', f'https://login.microsoftonline.com/{tenant_id}/oauth2/token')
 
# curated_storage_account = f"ingest00external{env}"

# silver_container = "external-csv"
 
# dbutils.fs.ls(f"abfss://{silver_container}@{curated_storage_account}.dfs.core.windows.net/ReferenceData/")
 

In [0]:
# %sql
# select *, size(tempCaseStatusDetails) as tempCaseStatusDetails_length from hive_metastore.ariadm_arm_fta.stg_statusdetail_data
# -- where CaseNo = 'CC/00009/2005'
# order by tempCaseStatusDetails_length desc
# limit 10

In [0]:
# %sql
# select * from hive_metastore.ariadm_arm_fta.silver_list_detail

In [0]:
# %sql
# select * from hive_metastore.ariadm_arm_fta.silver_transaction_detail

In [0]:
# %sql
# select currentStatus, KeyDate, * from hive_metastore.ariadm_arm_uta.silver_status_detail
# where CaseNo = 'CC/00009/2005'
# order by statusId desc